# Practica 2

### Maestria en Computo aplicado
#### *Topicos de la industria I*

**Nombre:**  Efren Del Real Frias  
**e-mail:**  efren.delreal8824@alumnos.udg.mx

**Fecha** Abril 01 del 2022

## MODULES

In [19]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

from google.colab import files

import os

## Working directory

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
!ls

drive  sample_data


In [18]:
# Change working directory path

%cd /content/drive/MyDrive/1_Master/2_Semester/1_TI1/0_Git/220326_Practica2/

/content/drive/MyDrive/1_Master/2_Semester/1_TI1/0_Git/220326_Practica2


In [21]:
# Keeps current working directory path
cwd: str = os.getcwd()
print(cwd)

/content/drive/MyDrive/1_Master/2_Semester/1_TI1/0_Git/220326_Practica2


In [32]:
trajPath: str = os.path.join(cwd, 'trajectories','')
print(trajPath)

/content/drive/MyDrive/1_Master/2_Semester/1_TI1/0_Git/220326_Practica2/trajectories/


## CLASSES

In [ ]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

### 1 turning_angle

In [2]:
###############################################################################################
# Turning angle
###############################################################################################
def turning_angle(vec_a, vec_b, vec_c):
    """
    Arguments:
        vec_a: First detection coordinates
        vec_b: Second detection coordinates
        vec_c: Third detection coordinates
    Returns:
        theta: Turning angle 
    """
    ab = vec_b-vec_a
    norm_ab = np.linalg.norm(ab)
    
    bc = vec_c-vec_b
    norm_bc = np.linalg.norm(bc)

    dot_p = np.dot(ab, bc)
    
    cross_p = np.cross(ab, bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1
    
    cos_theta = dot_p/(norm_ab*norm_bc+np.finfo(float).eps)
    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta

### 2 compute_path_lenght

In [3]:
###############################################################################################
# Compute Path Lenght
###############################################################################################
def compute_path_lenght(rw_2d_df):
  """
  Arguments:
        rw_2d_df: pd.DataFrame - Random Walk pandas DataFrame size(rows = n, columns = 2)

  Returns:
        pl:       np.array     - Path Lenght numpy array size(rows = n, columns = 1)

  See:
        https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.euclidean.html
  """
  disEuclidean = np.array([ distance.euclidean( rw_2d_df.iloc[i - 1], rw_2d_df.iloc[i])
                            for i in range(1, rw_2d_df.shape[0]) ])
  pl = np.cumsum(disEuclidean)

  return pl

### 3 mean_squared_displacement

$$
MSD = \frac{1}{N - n} \sum_{i=1}^{N-n}(r_{i+n} -r_{i})^2
$$

where $n = 1,...., N - 1$

In [43]:
###############################################################################################
# Compute Path Lenght
###############################################################################################
def mean_squared_displacement(rw_2d_df, tau: int) -> float:
  """
  Arguments:
        rw_2d_df: pd.DataFrame - Random Walk pandas DataFrame size(rows = n, columns = 2)
        tau:      integer      - iterable number to displace two points (called "n" in above equation)
  Returns:
        msd:      float        - mean squared displacement value
  """
  displacement = np.array([ distance.euclidean( rw_2d_df.iloc[i - tau], rw_2d_df.iloc[i] ) 
                            for i in range(tau, rw_2d_df.shape[0]) ])
  
  msd: float = np.mean(np.power(displacement, 2))
  
  return msd


### 4 levy_compute_step_lenght

In [49]:
###############################################################################################
# Compute Path Lenght
###############################################################################################
def levy_compute_step_lenght(levy_df):
  """
  Arguments:
        levy_df:                    pd.DataFrame - 
        Levy Walk pandas DataFrame size(rows = n, columns = 2)

  Returns:
        (aux_ta_Levy, aux_sl_Levy):  Tuple(np.array, np.array) - 
        Levy angle and Levy step lengh 
  """
  # Define Local Variables
  theta:        float = 0.0        
  prev_theta:   float = (3 * np.pi)  #
  prev_index:     int = 0
  cal_vel:       bool = True         # Calculate Velocity
  velocity:     float = 0.0

  # aux to store turning angles
  aux_ta_Levy = np.empty(shape=(0))
  # aux to store step-lengths
  aux_sl_Levy = np.empty(shape=(0))

  # Start to calculate over all DataFrame row values
  for index, _ in levy_df[1:-1].iterrows():

      a = levy_df.iloc[index - 1]
      b = levy_df.iloc[index]
      c = levy_df.iloc[index + 1]

      theta = turning_angle(a, b, c)
      
      # Ignore 0.0 since is still in the same direction
      if (theta != 0.0):

        if (prev_theta !=  theta):
          
          # Add to the end the new direction
          aux_ta_Levy = np.concatenate( (aux_ta_Levy, [theta]), axis=0)

          # Gets path lenght - This value includes velocity
          pl = distance.euclidean( levy_df.iloc[prev_index] , levy_df.iloc[index] )

          # Add to the end new path lenght
          pl = np.round(pl)
          aux_sl_Levy = np.concatenate ( (aux_sl_Levy, [pl]) , axis=0)


          # Keeps current index and angle to the next step
          prev_index = index
          prev_theta = theta
      else:
        # theta == 0.0
        if cal_vel:
          # Gets velocity from two close steps
          velocity = distance.euclidean( levy_df.iloc[prev_index] , levy_df.iloc[index] )
          
          # Clean in order to only execute one time
          cal_vel = False

  
  if not cal_vel:
    # We need to divite over the velocity in order
    # to get Levy step lenght
    aux_sl_Levy = aux_sl_Levy / velocity

  return aux_ta_Levy, aux_sl_Levy



## Actividad 1: Path Length - (BM1 vs BM2 vs CRW)

* Cargar trayectorias en **Pandas** Data Frame
* Calcular métrica utilizando exclusivamente funciones de NumPy
* Guardar métricas en **Pandas** Data Frame
* Visualizar con **plotly**

In [33]:
# Load existing trajectories
# BM speed = 3
BM_2d_df_3 = pd.read_csv(f'{trajPath}brownian_3.csv')

# Load existing trajectories
# BM speed = 6
BM_2d_df_6 = pd.read_csv(f'{trajPath}brownian_6.csv')

# Load existing trajectories
CRW_2d_df_9 = pd.read_csv(f'{trajPath}crw_6_9.csv')

In [34]:
# Compute path length
## start - Add your code here
pl_BM_3   = compute_path_lenght(BM_2d_df_3)
pl_BM_6   = compute_path_lenght(BM_2d_df_6)
pl_CRW_9  = compute_path_lenght(CRW_2d_df_9)
## end - Add your code here

In [35]:
# Plotting
# Init figure
fig_path_length = go.Figure()

# First trace BM1
## start - Add your code here
fig_path_length.add_trace(
    go.Scatter(
        x=np.arange(len(pl_BM_3)),
        y=pl_BM_3,
        marker=dict(size=2),
        line = dict(width=2),
        mode = 'lines',
        name = 'path_length_BM_3',
        showlegend = True
))
## end - Add your code here


# Second trace BM2
## start - Add your code here
fig_path_length.add_trace(
    go.Scatter(
        x=np.arange(len(pl_BM_6)),
        y=pl_BM_6,
        marker=dict(size=2),
        line = dict(width=8),
        mode = 'lines',
        name = 'path_length_BM_6',
        showlegend = True
))    
## end - Add your code here


# Third trace CRW
## start - Add your code here
fig_path_length.add_trace(
    go.Scatter(
        x=np.arange(len(pl_CRW_9)),
        y=pl_CRW_9,
        marker=dict(size=2),
        line = dict(width=2),
        mode = 'lines',
        name = 'path_length_CRW_9',
        showlegend = True
))    
## end - Add your code here


fig_path_length.show()

## Actividad 2: Mean Squared Displacement - (Brownian vs CRW)

* Cargar trayectorias en **Pandas** Data Frame
* Guardar metricas en **Pandas** Data Frame
* Visualizar con **plotly**

In [36]:
# Load existing trajectories
# BM speed = 6
BM_2d_df_6 = pd.read_csv(f'{trajPath}brownian_6.csv')

# Load existing trajectories
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv(f'{trajPath}crw_6_9.csv')

In [45]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter3d(x = BM_2d_df_6.x_pos,
                 y = BM_2d_df_6.y_pos,
                 z = BM_2d_df_6.index,
                 marker = dict(size=2),
                 line = dict(color='blue', width=2),
                 mode = 'lines',
                 name = 'BM 2d',
                 showlegend = True))


fig_3d.add_trace(
    go.Scatter3d(x = CRW_2d_df_9.x_pos,
                 y = CRW_2d_df_9.y_pos,
                 z = CRW_2d_df_9.index,
                 marker = dict(size=2),
                 line = dict(color='red', width=2),
                 mode = 'lines',
                 name = 'CRW 2d',
                 showlegend = True))

fig_3d.show()

In [46]:
# Empty MSD_BM
MSD_BM = np.empty(shape=(0))

# MSD for BM_2d_df_6
for tau in range(1,BM_2d_df_6.shape[0]):
    ## start - Add your code here
    MSD_BM = np.concatenate( (MSD_BM, [mean_squared_displacement(BM_2d_df_6, tau)]), axis=0)
    ## end - Add your code here

# Empty MSD_CRW
MSD_CRW = np.empty(shape=(0))
# MSD for CRW_2d_df_9
for tau in range(1,CRW_2d_df_9.shape[0]):
    ## start - Add your code here
    MSD_CRW = np.concatenate( (MSD_CRW, [mean_squared_displacement(CRW_2d_df_9, tau)]), axis=0)
    ## end - Add your code here
    
# Save metrics to Dataframe
## start - Add your code here

met_df = pd.DataFrame(
    data = np.concatenate( (
        np.reshape(MSD_BM   , (MSD_BM.shape[0]  , 1)),    #
        np.reshape(MSD_CRW  , (MSD_CRW.shape[0] , 1))),   #
        axis=1),                                          #
    columns = ['MSD_BM','MSD_CRW'])  
## end - Add your code here

In [ ]:
# Write to csv
## start - Add your code here
csvFile_name: str = 'met1_df.csv'
met_df.to_csv(f'./{csvFile_name}', index=False)
files.download(f'./{csvFile_name}')
## end - Add your code here

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [47]:
# Init figure
fig_path_length = go.Figure()

# first trace MSD_BM
## start - Add your code here
fig_path_length.add_trace(
    go.Scatter(
        x=met_df.index,
        y=met_df.MSD_BM,
        marker=dict(size=2),
        line = dict(width=2),
        mode = 'lines',
        name = 'MSD BM 6',
        showlegend = True
))       
## end - Add your code here


# Second trace MSD_CRW
## start - Add your code here
fig_path_length.add_trace(
    go.Scatter(
        x=met_df.index,
        y=met_df.MSD_CRW,
        marker=dict(size=2),
        line = dict(width=2),
        mode = 'lines',
        name = 'MSD CRW 6 c=0.9',
        showlegend = True
))         
## end - Add your code here


fig_path_length.show()

## Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada)

* Generar CRWs con dos exponentes diferentes
* Guardar trayectorias en Pandas Data Frame
* Obtener Turning-angle distribution
* Guardar metricas en **Pandas** Data Frame
* Comparar en gráfica distribución origen vs distribución observada (Histograma)
* Visualizar con **plotly**

In [38]:
# Load existing trajectories
# CRW speed = 6, 
# wrapcauchy [c = 0.6]
CRW_2d_df_6 = pd.read_csv(f'{trajPath}crw_6_6.csv')

# Load existing trajectories
# CRW speed = 6, 
# wrapcauchy [c = 0.9]
CRW_2d_df_9 = pd.read_csv(f'{trajPath}crw_6_9.csv')

In [ ]:
# aux to store turning angles
aux_ta_CRW_2d_df_6 = np.empty(shape=(0))


# Iterate over trajectory compute turning angles
for index, row in CRW_2d_df_6[1:-1].iterrows():
    ## start - Add your code here
    a = CRW_2d_df_6.iloc[index - 1]
    b = CRW_2d_df_6.iloc[index]
    c = CRW_2d_df_6.iloc[index + 1]
    aux_ta_CRW_2d_df_6 = np.concatenate( (aux_ta_CRW_2d_df_6, [turning_angle(a, b, c)]), axis=0)
    ## end - Add your code here

# aux to store turning angles
aux_ta_CRW_2d_df_9= np.empty(shape=(0))


# Iterate over trajectory compute turning angles
for index, row in CRW_2d_df_9[1:-1].iterrows():
    ## start - Add your code here
    a = CRW_2d_df_9.iloc[index - 1]
    b = CRW_2d_df_9.iloc[index]
    c = CRW_2d_df_9.iloc[index + 1]
    aux_ta_CRW_2d_df_9 = np.concatenate( (aux_ta_CRW_2d_df_9, [turning_angle(a, b, c)]), axis=0)
    ## end - Add your code here

    
# Save to pandas DF
## start - Add your code here
met3_df = pd.DataFrame( 
    data=np.concatenate( (
        np.reshape(aux_ta_CRW_2d_df_6, (aux_ta_CRW_2d_df_6.shape[0], 1)),
        np.reshape(aux_ta_CRW_2d_df_9, (aux_ta_CRW_2d_df_9.shape[0], 1)) ), 
        axis=1), 
    columns = ['TA_CRW_6','TA_CRW_9'])      
## end - Add your code here


# Write to csv
## start - Add your code here
csvFile_name: str = 'met3_df.csv'
met3_df.to_csv(f'./{csvFile_name}', index=False)
files.download(f'./{csvFile_name}')    
## end - Add your code here

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Check documentation
# https://plotly.com/python/histograms/

# PLot histogram
fig_met_df_3 = go.Figure()


# Histogram turning angle CRW_2d_df_6
## start - Add your code here
fig_met_df_3.add_trace(
    go.Histogram(
        x=met3_df.TA_CRW_6,
        xbins=dict(size=np.pi/100),
        name='Obseved_0.6',
        opacity=0.75,
        histnorm='probability density',
        showlegend=True
        ))
## end - Add your code here


# Histogram turning angle CRW_2d_df_9
## start - Add your code here
fig_met_df_3.add_trace(
    go.Histogram(
        x=met3_df.TA_CRW_9,
        xbins=dict(size=np.pi/100),
        name='Obseved_0.9',
        opacity=0.75,
        histnorm='probability density',
        showlegend=True
        ))    
## end - Add your code here


# Add origin distributions
## start - Add your code here
n_samples:  int = met3_df.shape[0]
WCDLL:    float = 0.0               # Wrap Cauchy Domain Lower Limit
WCDUL:    float = (2.0 * np.pi)     # Wrap Cauchy Domain Upper Limit

n_half_samples  = int(n_samples / 2.0)
 
CRW_exponent    = (0.6, 0.9) 
CRW_color_plot  = ('blue', 'red')


x_domain      = np.linspace(WCDLL, WCDUL, n_samples)
x_domain_plot = np.linspace(-np.pi, np.pi, n_samples)

for idx, iCRWe in enumerate(CRW_exponent):

  # Gets Wrap Cauchy Probability desnsity function 
  wrapcauchy_pdf = wrapcauchy.pdf(x_domain, iCRWe)

  # Swap probability values in order to get a mirror view in 0 position
  wrapcauchy_pdf = np.concatenate(
      (
          wrapcauchy_pdf[n_half_samples: ],
          wrapcauchy_pdf[ :n_half_samples]
      ),
      axis=0
  )
  
  # Add
  fig_met_df_3.add_trace(
      go.Scatter(
          x = x_domain_plot,
          y = wrapcauchy_pdf,
          marker = dict(size=2),
          line = dict(color=CRW_color_plot[idx],width=2),
          mode = 'lines',
          name = f'cauchy_{iCRWe}',
          showlegend = True ) )       
## end - Add your code here


fig_met_df_3.show()

## Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada)

* Generar Levy Flight
* Guardar trayectorias en **Pandas** Data Frame
* Guardar metricas en **numpy** array
* Obtener Step-length distribution
* Comparar en gráfica distribución origen vs distribución observada
* Visualizar con **plotly**

In [39]:
# Load existing trajectories
# Levy speed = 6
# levy_stable [alpha=1.0, beta=1.0, loc=3.0]
Levy_2d_df_1 = pd.read_csv(f'{trajPath}levy_6_1.csv')

# Load existing trajectories
# Levy speed = 6
# levy_stable [alpha=0.7, beta=1.0, loc=3.0]
Levy_2d_df_7 = pd.read_csv(f'{trajPath}levy_6_7.csv')

In [41]:
# aux to store turning angles
aux_ta_Levy_2d_df_1 = np.empty(shape=(0))
# aux to store step-lengths
aux_sl_Levy_2d_df_1 = np.empty(shape=(0))

## start - Add your code here
_, aux_sl_Levy_2d_df_1 = levy_compute_step_lenght(Levy_2d_df_1)
## end - Add your code here


# aux to store turning angles
aux_ta_Levy_2d_df_7 = np.empty(shape=(0))
# aux to store step-lengths
aux_sl_Levy_2d_df_7 = np.empty(shape=(0))

## start - Add your code here
_, aux_sl_Levy_2d_df_7 = levy_compute_step_lenght(Levy_2d_df_7)
## end - Add your code here

In [48]:
# Local variables
n_samples:          int = 100
std_motion_steps:   int = 5
beta:             float = 1.0
dMinValue:          int = 0       # domain min value
dMaxValue:          int = 50      # domain max value
alpha_t                 = (1.0, 0.7)
CRW_color_plot          = ('blue', 'red') # colors tuple

x_domain      = np.linspace( dMinValue, dMaxValue, n_samples)


# PLot histogram
fig_met_df_4 = go.Figure()

# Histogram step-length Levy_2d_df_1
## start - Add your code here
fig_met_df_4.add_trace(
    go.Histogram(
        x=aux_sl_Levy_2d_df_1,
        xbins=dict(start= dMinValue, end=dMaxValue, size=1),
        name='Obseved_alpha=1.0_beta=1.0',
        opacity=0.75,
        histnorm='probability density',
        showlegend=True
        ))    
## end - Add your code here


# Histogram step-length Levy_2d_df_7
## start - Add your code here
fig_met_df_4.add_trace(
    go.Histogram(
        x=aux_sl_Levy_2d_df_7,
        xbins=dict(start= dMinValue, end=dMaxValue, size=1),
        name='Obseved_alpha=0.7_beta=1.0',
        opacity=0.75,
        histnorm='probability density',
        showlegend=True
        ))      
## end - Add your code here


# Add origin distributions
## start - Add your code here
for idx, ialpha in enumerate(alpha_t):
  levy_stable_pdf = levy_stable.pdf(x_domain, alpha=ialpha, beta=beta, loc=std_motion_steps)

  fig_met_df_4.add_trace(
      go.Scatter(
          x = x_domain,
          y = levy_stable_pdf,
          marker = dict(size=2),
          line = dict(color=f'{CRW_color_plot[idx]}',width=2),
          mode = 'lines',
          name = f'Levy_alpha={ialpha}',
          showlegend = True ) )        
## end - Add your code here

fig_met_df_4.show()